<a href="https://colab.research.google.com/github/mablus/EFRAIM/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import files
uploaded_file = files.upload()
pdf_filename = list(uploaded_file.keys())[0]
print(pdf_filename)

Saving FP CEU9908 POACWB 16022022.pdf to FP CEU9908 POACWB 16022022.pdf
FP CEU9908 POACWB 16022022.pdf


In [23]:
pdf_path = pdf_filename

In [24]:
!pip install PyMuPDF
!sudo apt-get install -y libcrypt-dev
!pip install PyPDF2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcrypt-dev is already the newest version (1:4.4.27-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [34]:
import re
from google.colab import files
import PyPDF2

def recognize_flight_plan(text):
    keywords = ['flight number', 'origin', 'destination', 'alternate', 'orgn', 'org', 'dst' ,'dest', 'altrn', 'altn', 'mzfw', 'tow','ETOW','ELDW','EZFW','EPLD','MLDW','MZFW']

    for keyword in keywords:
        if re.search(keyword, text, re.IGNORECASE):
            return True
    return False

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# Specify the path to your PDF file
pdf_path = '/content/' + pdf_filename
#pdf_path = input("Enter the path to the PDF file: ") caminho do site
pdf_text = extract_text_from_pdf(pdf_path)

if recognize_flight_plan(pdf_text):
    print("Recognized as a flight plan.")
else:
    print("Not recognized as a flight plan.")

def recognize_flight_plan(text):
    keywords = ['flight number', 'origin', 'destination', 'alternate', 'orgn', 'dest', 'altrn', 'altn', 'mzfw', 'tow']

    for keyword in keywords:
        if re.search(keyword, text, re.IGNORECASE):
            return True
    return False

def extract_fuel_data(text):
    fuel_data = {}

    fuel_pattern = r"TRIP (\d+) (\d+\.\d+) (\d+) (\d+) ETOW (\d+) MTOW (\d+)\s+ALTN/(\w{4}) (\d+) (\d+\.\d+) (\d+) (\d+) ELDW (\d+) MLDW (\d+)\s+RESERVE \(10%\) (\d+) (\d+\.\d+) EZFW (\d+) MZFW (\d+)\s+HOLDING (\d+) (\d+\.\d+) EPLD (\d+)\s+MIN FUEL REQ (\d+) (\d+\.\d+) OPNLWT (\d+)\s+CONTINGENCY"

    fuel_match = re.search(fuel_pattern, text)
    if fuel_match:
        fuel_data['trip_dist'] = int(fuel_match.group(1))
        fuel_data['trip_time'] = float(fuel_match.group(2))
        fuel_data['trip_fuel'] = int(fuel_match.group(3))
        fuel_data['planned_fuel'] = int(fuel_match.group(4))
        fuel_data['etow'] = int(fuel_match.group(5))
        fuel_data['mtow'] = int(fuel_match.group(6))
        fuel_data['altn'] = fuel_match.group(7)
        fuel_data['altn_dist'] = int(fuel_match.group(8))
        fuel_data['altn_time'] = float(fuel_match.group(9))
        fuel_data['altn_fuel'] = int(fuel_match.group(10))
        fuel_data['altn_planned_fuel'] = int(fuel_match.group(11))
        fuel_data['eldw'] = int(fuel_match.group(12))
        fuel_data['mldw'] = int(fuel_match.group(13))
        fuel_data['reserve_fuel'] = int(fuel_match.group(14))
        fuel_data['reserve_time'] = float(fuel_match.group(15))
        fuel_data['ezfw'] = int(fuel_match.group(16))
        fuel_data['mzfw'] = int(fuel_match.group(17))
        fuel_data['holding_dist'] = int(fuel_match.group(18))
        fuel_data['holding_time'] = float(fuel_match.group(19))
        fuel_data['epld'] = int(fuel_match.group(20))
        fuel_data['min_fuel_req'] = int(fuel_match.group(21))
        fuel_data['min_fuel_time'] = float(fuel_match.group(22))
        fuel_data['opnlwt'] = int(fuel_match.group(23))

    return fuel_data
    """esta parte aqui poderá ser subsituida por uma media do fuel flow or media
    historica do consumo de fuel do voo e model da craft. Inicialmente apenas averigua-se
    se valor é Osmar ou não dando assim permissão para seguir ou não"""

# Function to extract text from PDF
"""def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# Get the PDF file path as input from the user
pdf_path = input("Enter the path to the PDF file: ")
pdf_text = extract_text_from_pdf(pdf_path)"""

if recognize_flight_plan(pdf_text):
    print("No fuel void or empty.")

    fuel_data = extract_fuel_data(pdf_text)

    # Check if any extracted number is less than or equal to 0
    has_invalid_numbers = any(value <= 0 for value in fuel_data.values())

    if has_invalid_numbers:
        print("Invalid numbers detected. Check your flight plan.")
    else:
        print("Valid flight plan.")

else:
    print("Not recognized as a flight plan.")

Recognized as a flight plan.
No fuel void or empty.
Valid flight plan.


In [35]:
import fitz
import re

# Caminho para o arquivo PDF
pdf_path = '/content/' + pdf_filename

# Função para extrair texto do PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    pdf_document = fitz.open(pdf_path)  # Abrir o documento PDF
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        text += page.get_text()  # Extrair texto da página atual
    return text

# Extrair texto do PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Usar regex para encontrar as seções desejadas entre "AWY" e "FT"
regex_pattern = r"AWY(.*?)FT"
matches = re.findall(regex_pattern, pdf_text, re.DOTALL)

# Imprimir as correspondências encontradas
for match in matches:
    print("Correspondência encontrada:", "AWY" + match + ",")  # Mantém a pontuação


Correspondência encontrada: AWY    CPT   MCS  FLT  ZD  ZT  ETA  ATA  DSTR  WIND   TAS GRS  FUEBO
Page 1
MSA    FREQ       SR       AC                  TEMP              MFR
..    NELOX  123  ..  001 0/01 ..../.... 0292   .. .  ..  ..  000046
...               1       0/01                 P24            019585
UM424 RAXUS  016  ..  026 0/04 ..../.... 0266   .. .  ..  ..  001585
043               2       0/05                 P06            018046
UM424 PALEL  017  ..  005 0/01 ..../.... 0261   .. .  ..  ..  001869
043               1       0/06                 P02            017762
UM424 CXS    016  ..  016 0/03 ..../.... 0245   .. .  ..  ..  002858
043   112.3       1       0/09                 M15            016773
UZ104 ARTIL  047  ..  026 0/04 ..../.... 0219   .. .  ..  ..  004128
070               6       0/13                 M29            015503
UZ104 TOC    047  ..  043 0/04 ..../.... 0176   .. .  ..  ..  005142
074               6       0/17                 M45            014489

In [36]:
import re

# Seu texto
texto = match

# Usar regex para encontrar os CPTs entre 'TEMP' e 'ELEV'
regex_pattern = r"MFR(.*?)ELEV"
matches = re.search(regex_pattern, texto, re.DOTALL)

if matches:
    correspondencia = matches.group(1)
    cpts = re.findall(r"\b[A-Z]+\b", correspondencia)
    print("CPTs encontrados:", cpts)
else:
    print("Nenhuma correspondência encontrada.")

import re

text = match

# Define o padrão de início e fim
start_pattern = "MFR"
end_pattern = "ELEV"

# Extrai a parte relevante do texto
relevant_text = re.search(start_pattern + r"(.*?)" + end_pattern, text, re.DOTALL).group(1)

# Define o padrão para extrair o número SR
sr_pattern = r"\d+\s+\d+\s+(\d+)\s+\d+/\d+"

# Encontra todas as correspondências do padrão SR no texto relevante
sr_numbers = re.findall(sr_pattern, relevant_text)

# Filtra os números SR com mais de 2 dígitos e que não começam com 0
filtered_sr_numbers = [number for number in sr_numbers if len(number) <= 2 and not number.startswith("0")]

# Combina os CPTs e números SR em pares
cpt_sr_pairs = list(zip(cpts, filtered_sr_numbers))

# Inicializa uma lista para armazenar alertas
alertas = []

# Verifica se os números SR são maiores que 3 e gera alertas
for cpt, sr in cpt_sr_pairs:
    sr_int = int(sr)
    if sr_int > 2:
        alerta = f"Shear maior que três no fixo {cpt}: {sr_int}"
        alertas.append(alerta)

# Imprime os alertas, se houver
if alertas:
    print("Alertas:")
    for alerta in alertas:
        print(alerta)
else:
    print("Nenhum alerta de shear")

CPTs encontrados: ['NELOX', 'RAXUS', 'PALEL', 'CXS', 'ARTIL', 'TOC', 'VAPAP', 'UMGAK', 'TOD', 'SAVLU', 'SBCT']
Alertas:
Shear maior que três no fixo PALEL: 6
Shear maior que três no fixo CXS: 6
Shear maior que três no fixo ARTIL: 6
Shear maior que três no fixo TOC: 6
Shear maior que três no fixo VAPAP: 5
